# Nygaard TDM

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
U_h0 = 10
C_T = 0.8
D = 100
s_x = 10

In [8]:
def calculate_wind_speed(x1, x2, x3, z_h, D, C_T, s_x, s_y, U_h0):

    k = 0.4 # Von Karman constant [-]
    z_0lo = 0.0001 # surface rughness length [m]

    ct = (np.pi * C_T * D**2)/(8 * s_x*D * s_y*D)

    nu_star = 28 * np.sqrt(ct)

    beta =  nu_star / (1 + nu_star)

    z0_WF = z_h * (1 + D/(2 * z_h))**beta * np.exp(-k * (ct + (k/np.log((z_h / z_0lo) * (1 - D/(2 * z_h))**beta))**2)**-0.5)

    x = np.linspace(x1, x3, x3)

    h1 = np.full_like(x, np.nan, dtype=float)
    h2 = np.full_like(x, np.nan, dtype=float)

    h1[x > x1] = z_h + ((x[x > x1]) / z0_WF)**(4/5)
    h2[x > x2] = ((x[x>x2]-x2)/z0_WF)**(4/5)

    uw = np.full_like(x, np.nan, dtype=float)

    idx1 = h1 >= z_h
    uw[idx1] = U_h0 * (np.log(h1[idx1] / z_0lo) * np.log(z_h / z0_WF)) / (np.log(h1[idx1] / z0_WF) * np.log(z_h/z_0lo))

    idx2 = (x > x2) & (h2 >= z_h)
    uw[idx2] = U_h0 * (np.log(h1[idx2] / z_0lo) * np.log(h2[idx2] / z0_WF)) / (np.log(h1[idx2] / z0_WF) * np.log(h2[idx2] / z_0lo))

    return x, uw

# Alex's firkanter :)

In [10]:
h_farm = D
w_farm = s_x*D * 5
l_farm = w_farm # Midlertidigt pga. kvadrat
A_0 = w_farm * h_farm
x_meas = 400*D

x, u_tdm = calculate_wind_speed(0, l_farm, x_meas, 100, D, 0.9, 10, 10, U_h0)
u_s = (U_h0 - u_tdm[-1]) / U_h0
print('u_s', u_s)

u_s 0.004744711532551982


In [11]:
kx = (- h_farm/4 - w_farm/4 + (np.sqrt(h_farm**2 + 2*A_0 + w_farm**2 - 4*A_0 + 4*(A_0 * (C_T / (2*(u_s - u_s**2))))))/4)/x_meas
print('kx:', round(kx,2), '[m/m]')

kx: 0.06 [m/m]


# Save to CSV

In [12]:
k = int(kx*x_meas)
y_meas = np.linspace(0 - k, w_farm + k, w_farm+(2*k)+1)
U = np.full_like(y_meas, u_tdm[-1])

In [13]:
data = pd.DataFrame({
    "y_wake": y_meas / D,
    "U_norm": U / U_h0,
})
data.to_csv(f"../ArbitraryLayouts/ValidationData/mREC{int(x_meas / D)}D.csv", index=False)